In [4]:
import os
import random
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

tfk = tf.keras
tfkl = tf.keras.layers

print(tf.__version__)

2.10.0


In [5]:
# Random seed for reproducibility
SEED = 42

random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.compat.v1.set_random_seed(SEED)

In [6]:
# All the categories to classify the dataset
LABELS = ['Species1',              # 0
          'Species2',          # 1
          "Species3",             # 2
          "Species4",          # 3
          "Species5",            # 4
          "Species6",             # 5
          "Species7",         # 6
          "Species8"]             # 7

In [7]:
# Model hyperparameters
INPUT_SHAPE = (96, 96, 3)
IMAGE_SIZE = (INPUT_SHAPE[0], INPUT_SHAPE[1])
EPOCHS = 100
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
NUM_CLASSES = len(LABELS)
MODEL_NAME = "cnn"

In [8]:
# Paths
ROOT_PATH = 'data5/data5' # The name of the directory cotaining the splitted dataset, it should be in the same directory of this notebook
TRAINING_DIR = os.path.join(ROOT_PATH, 'training')
VALIDATION_DIR = os.path.join(ROOT_PATH, 'validation')


In [9]:
# Constructor
train_data_gen = ImageDataGenerator(
    rotation_range=50,
    height_shift_range=5,
    width_shift_range=5,
    zoom_range=0.5,
    shear_range = 0.1,
    brightness_range=[0.4, 1.5],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect',
    rescale=1/255.
)

# Generator
train_gen = train_data_gen.flow_from_directory(
    directory=TRAINING_DIR,
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED
)

Found 2656 images belonging to 8 classes.


In [10]:
# Constructor
valid_data_gen = ImageDataGenerator(
    rescale=1/255.
)

# Generator
valid_gen = valid_data_gen.flow_from_directory(
    directory=VALIDATION_DIR,
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED
)

Found 886 images belonging to 8 classes.


In [11]:
# Download and plot the EfficientNet B0 model
supernet = tfk.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=INPUT_SHAPE
)

# Plot the model
supernet.summary()
tfk.utils.plot_model(supernet)

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 rescaling (Rescaling)          (None, 96, 96, 3)    0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 96, 96, 3)    7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)        (None, 96, 96, 3)    0           ['normalization[0][0]']          
                                                                                     

                                                                                                  
 block2b_expand_bn (BatchNormal  (None, 24, 24, 144)  576        ['block2b_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block2b_expand_activation (Act  (None, 24, 24, 144)  0          ['block2b_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block2b_dwconv (DepthwiseConv2  (None, 24, 24, 144)  1296       ['block2b_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block2b_b

 block3b_dwconv (DepthwiseConv2  (None, 12, 12, 240)  6000       ['block3b_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block3b_bn (BatchNormalization  (None, 12, 12, 240)  960        ['block3b_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block3b_activation (Activation  (None, 12, 12, 240)  0          ['block3b_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block3b_se_squeeze (GlobalAver  (None, 240)         0           ['block3b_activation[0][0]']     
 agePoolin

 )                                                                                                
                                                                                                  
 block4b_se_squeeze (GlobalAver  (None, 480)         0           ['block4b_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block4b_se_reshape (Reshape)   (None, 1, 1, 480)    0           ['block4b_se_squeeze[0][0]']     
                                                                                                  
 block4b_se_reduce (Conv2D)     (None, 1, 1, 20)     9620        ['block4b_se_reshape[0][0]']     
                                                                                                  
 block4b_se_expand (Conv2D)     (None, 1, 1, 480)    10080       ['block4b_se_reduce[0][0]']      
          

 block5a_se_reshape (Reshape)   (None, 1, 1, 480)    0           ['block5a_se_squeeze[0][0]']     
                                                                                                  
 block5a_se_reduce (Conv2D)     (None, 1, 1, 20)     9620        ['block5a_se_reshape[0][0]']     
                                                                                                  
 block5a_se_expand (Conv2D)     (None, 1, 1, 480)    10080       ['block5a_se_reduce[0][0]']      
                                                                                                  
 block5a_se_excite (Multiply)   (None, 6, 6, 480)    0           ['block5a_activation[0][0]',     
                                                                  'block5a_se_expand[0][0]']      
                                                                                                  
 block5a_project_conv (Conv2D)  (None, 6, 6, 112)    53760       ['block5a_se_excite[0][0]']      
          

                                                                                                  
 block5c_project_bn (BatchNorma  (None, 6, 6, 112)   448         ['block5c_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block5c_drop (Dropout)         (None, 6, 6, 112)    0           ['block5c_project_bn[0][0]']     
                                                                                                  
 block5c_add (Add)              (None, 6, 6, 112)    0           ['block5c_drop[0][0]',           
                                                                  'block5b_add[0][0]']            
                                                                                                  
 block6a_expand_conv (Conv2D)   (None, 6, 6, 672)    75264       ['block5c_add[0][0]']            
          

                                                                  'block6a_project_bn[0][0]']     
                                                                                                  
 block6c_expand_conv (Conv2D)   (None, 3, 3, 1152)   221184      ['block6b_add[0][0]']            
                                                                                                  
 block6c_expand_bn (BatchNormal  (None, 3, 3, 1152)  4608        ['block6c_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block6c_expand_activation (Act  (None, 3, 3, 1152)  0           ['block6c_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block6c_d

 ization)                                                                                         
                                                                                                  
 block7a_expand_activation (Act  (None, 3, 3, 1152)  0           ['block7a_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block7a_dwconv (DepthwiseConv2  (None, 3, 3, 1152)  10368       ['block7a_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block7a_bn (BatchNormalization  (None, 3, 3, 1152)  4608        ['block7a_dwconv[0][0]']         
 )                                                                                                
          

In [12]:

# Use the supernet as feature extractor
supernet.trainable = False

# Rebuild the classifier
inputs = tfk.Input(shape=INPUT_SHAPE)
x = supernet(inputs)
x = tfkl.GlobalAveragePooling2D()(x)
x = tfkl.Dense(256, kernel_initializer = tfk.initializers.GlorotUniform(SEED))(x)
x = tfkl.BatchNormalization()(x)
x = tfkl.ReLU()(x)
outputs = tfkl.Dense(
    NUM_CLASSES, 
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(SEED))(x)

# Connect input and output through the Model class
model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=LEARNING_RATE), metrics=['accuracy'])
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 efficientnetb0 (Functional)  (None, 3, 3, 1280)       4049571   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 256)               0     

In [15]:
for layer in supernet.layers[:162]:
    layer.trainable = False

In [16]:
# Compile the model
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=LEARNING_RATE), metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 efficientnetb0 (Functional)  (None, 3, 3, 1280)       4049571   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 256)               0     

In [17]:
class_weights = {0: 2.389358108108108, 
                 1: 0.8320588235294117, 
                 2: 0.8583131067961165, 
                 3: 0.8667279411764706, 
                 4: 0.8340212264150944, 
                 5: 1.9978813559322033, 
                 6: 0.8243006993006993, 
                 7: 0.8709975369458128}

In [18]:
callbacks = []

# Tensorboard
tensorboard = TensorBoard(log_dir="tb_logs/")
callbacks.append(tensorboard)
# Early Stopping
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10,
                                                restore_best_weights=True)
callbacks.append(es_callback)


#tuner = keras_tuner.RandomSearch(
#    build_model,
#    objective='val_accuracy',
#    max_trials=4)

#tuner.search(aug_train_gen, epochs=200, validation_data=(val_gen))
#model = tuner.get_best_models()[0]

# Train the model
model.fit(
    x=train_gen,
    batch_size=BATCH_SIZE,
    epochs=200,
    class_weight=class_weights,
    validation_data=valid_gen,
    callbacks= callbacks)


model.save("testing_x/EN")

Epoch 1/200
83/83 [==============================] - 112s 1s/step - loss: 2.1876 - accuracy: 0.1212 - val_loss: 2.0849 - val_accuracy: 0.1501
Epoch 2/200
83/83 [==============================] - 49s 595ms/step - loss: 2.1169 - accuracy: 0.1303 - val_loss: 2.0561 - val_accuracy: 0.1670
Epoch 3/200
29/83 [=========>....................] - ETA: 26s - loss: 2.1474 - accuracy: 0.1358

KeyboardInterrupt: 

In [ ]:
model = tfk.models.load_model('testing_x/EN')
model.summary()

In [ ]:
for layer in supernet.layers[:fine_tuning_locked]:
    layer.trainable = False

In [ ]:
# Compile the model
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

In [24]:
# Fine-tune the model
model.fit(
    x = aug_train_gen,
    batch_size = batch_size,
    epochs = epochs,
    validation_data=valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
)

NameError: name 'aug_train_gen' is not defined

In [ ]:
model.save('testing_x/ENT')